In [ ]:
%reset -f
import time
import pickle
import gym

# from stable_baselines3 import DDPG
# from stable_baselines3.common.env_checker import check_env
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import derivatives as d
import forward_helpers as fh
from chameleon import Chameleon

In [ ]:
n_elems = 50
steps = 1_000
chameleon = Chameleon(E=0.1, n_elems=n_elems, dt=1e-5)

In [ ]:
chameleon.with_grad

In [ ]:
chameleon.dt

In [ ]:
# E = 1, 10 works E = 0.1, 100 breaks
# active_stress = np.array(
#     [
#         0.42846936,
#         0.47040876,
#         0.53477369,
#         0.62156414,
#         0.7307801,
#         0.86242159,
#         1.0164886,
#         1.19298113,
#         1.39189918,
#         1.61324275,
#     ]
# )
active_stress = 10 * np.linspace(1, 10, n_elems)

In [ ]:
%%time
fh.forward_simulate(chameleon, active_stress, sim_steps=steps)

In [ ]:
chameleon.pos_f

In [ ]:
diffs = np.diff(chameleon.pos_f)
idxs = np.where(diffs < 0)

In [ ]:
idxs

In [ ]:
v_term = (chameleon.pos_f[-1] - chameleon.position_history[-2][-2]) / chameleon.dt

In [ ]:
v_term

In [ ]:
# chameleon.position_history
# abs_val_diffs = np.abs(diffs[idxs])
# len(chameleon.position_history)
# np.mean(abs_val_diffs)

In [ ]:
cham2 = Chameleon(E=10, n_elems=n_elems, dt=1e-5)
cham2.pos_0 = chameleon.position_history[-1]

In [ ]:
cham2.pos_f

In [ ]:
fh.forward_simulate(chameleon, active_stress, sim_steps=steps)

In [ ]:
chameleon.pos_f

In [ ]:
fh.forward_simulate(cham2, active_stress, sim_steps=steps)

In [ ]:
cham2.pos_f

In [ ]:
diffs2 = np.diff(cham2.pos_f)
idxs2 = np.where(diffs2 < 0)

In [ ]:
idxs2

In [ ]:
chame

# debugging diffs when elements come out of order 
The mean diff when I have 10,000 elements is 0.00029755194344014525

The mean diff when i have 1000 elements is 0.0028333580473896636

The mean diff when I have 100 elements is 0.018904419405243397

This is obvious. if i have more elements in the same volume then there will be less diff. I guess the encouraging thing is that it gets less messed up when I put in more elements. 

In [ ]:
plt.scatter(chameleon.pos_f, 0 * chameleon.pos_f)

In [ ]:
history = []
for i in range(len(chameleon.position_history)):
    x_p = chameleon.position_history[i]
    y = np.zeros(chameleon.n_elems)
    ar = np.vstack((x_p, y)).T
    history.append(ar)

with open("history.txt", "wb") as fp:
    pickle.dump(history, fp)